In [1]:
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

In [2]:
#filepath of the stored files

filepath = 'processed_data/'

In [3]:
#Load Form 990 files

Form_990_2021 = pd.read_csv(filepath+'2021_990_output.csv', header=0)
Form_990_2022 = pd.read_csv(filepath+'2022_990_output.csv', header=0)
Form_990_2023 = pd.read_csv(filepath+'2023_990_output.csv', header=0)

#Load Form 990PF Files

Form_990PF_2021 = pd.read_csv(filepath+'2021_990PF_output.csv', header=0)
Form_990PF_2022 = pd.read_csv(filepath+'2022_990PF_output.csv', header=0)
Form_990PF_2023 = pd.read_csv(filepath+'2023_990PF_output.csv', header=0)

In [4]:
#Consolidate the form 990 and 990PF results in respective files

Form_990_files = [Form_990_2021, Form_990_2022, Form_990_2023]
Form_990PF_files = [Form_990PF_2021, Form_990PF_2022, Form_990PF_2023]

Form_990 = pd.concat(Form_990_files) 
Form_990PF = pd.concat(Form_990PF_files) 

In [5]:
#Add form types to files

Form_990PF.insert(0, 'FormType', '990PF')
Form_990.insert(0, 'FormType', '990')

In [6]:
Form_990['CashGrantAmount'] = Form_990['CashGrantAmount'].fillna(0).astype('float')
Form_990[ 'NonCashGrantAmount'] = Form_990[ 'NonCashGrantAmount'].fillna(0).astype('float')
Form_990['GrantAmount'] = Form_990['CashGrantAmount'] + Form_990['NonCashGrantAmount']

In [7]:
Form_990.drop(columns=['RecipientIrsSection', 'CashGrantAmount', 'NonCashGrantAmount', 'MethodOfNonCashGrant', 'GrantPurpose'], axis=1, inplace=True)
Form_990PF.drop(columns=['RecipientFoundationStatus', 'GrantPurpose'], axis=1, inplace=True)

In [8]:
Form_990['RecipientEIN'] = Form_990['RecipientEIN'].fillna(0).astype('int')
Form_990['RecipientZIP'] = Form_990['RecipientZIP'].fillna(0).astype('int').astype('str').str[:5].astype('int')

Form_990PF['RecipientZIP'] = Form_990PF['RecipientZIP'].fillna(0).astype('int').astype('str').str[:5].astype('int')

In [9]:
Form_990PF.rename(columns={'AddressLine1': 'Address', 'TotalGrantsAndAllocations': 'GrantsAndAllocations', 'RecipientAddressLine1':'RecipientAddress'}, inplace=True)

In [10]:
Form_990 = Form_990.applymap(lambda x: x.upper().strip() if isinstance(x, str) else x)
Form_990PF = Form_990PF.applymap(lambda x: x.upper().strip() if isinstance(x, str) else x)

In [11]:
#Source: https://github.com/Nonprofit-Open-Data-Collective/mission-taxonomies/tree/main

industry_map = {
    "ART":	"Arts, Culture, and Humanities",
    "EDU":	"Education",
    "UNI":	"Universities",
    "ENV":	"Environment and Animals",
    "HEL":	"Health",
    "HOS":	"Hospitals",
    "HMS":	"Human Services",
    "IFA":	"International, Foreign Affairs",
    "PSB":	"Public, Societal Benefit",
    "REL":	"Religion Related",
    "MMB":	"Mutual/Membership Benefit",
    "UNU":	"Unknown, Unclassified"
    }

organization_map = {
    "RG":	"Regular Nonprofit",
    "AA":	"Alliance/Advocacy Organizations",
    "MT":	"Management and Technical Assistance",
    "PA":	"Professional Societies/Associations",
    "RP":	"Research Institutes and/or Public Policy Analysis",
    "MS":	"Monetary Support - Single Organization",
    "MM":	"Monetary Support - Multiple Organizations",
    "NS":	"Nonmonetary Support Not Elsewhere Classified (N.E.C.)"
    }

major_map = {
    "A01":"Alliances & Advocacy",
    "A02":"Management & Technical Assistance",
    "A03":"Professional Societies & Associations",
    "A05":"Research Institutes & Public Policy Analysis",
    "A11":"Single Organization Support",
    "A12":"Fund Raising & Fund Distribution",
    "A19":"Support N.E.C.",
    "A20":"Arts & Culture",
    "A23":"Cultural & Ethnic Awareness",
    "A24":"Folk Arts",
    "A25":"Arts Education",
    "A26":"Arts & Humanities Councils & Agencies",
    "A27":"Community Celebrations",
    "A30":"Media & Communications",
    "A31":"Film & Video",
    "A32":"Television",
    "A33":"Printing & Publishing",
    "A34":"Radio",
    "A40":"Visual Arts",
    "A50":"Museums",
    "A51":"Art Museums",
    "A52":"Children's Museums",
    "A53":"Folk Arts Museums</i>",
    "A54":"History Museums",
    "A56":"Natural History & Natural Science Museums",
    "A57":"Science & Technology Museums",
    "A60":"Performing Arts",
    "A61":"Performing Arts Centers",
    "A62":"Dance",
    "A63":"Ballet",
    "A65":"Theater",
    "A68":"Music",
    "A69":"Symphony Orchestras",
    "A6A":"Opera",
    "A6B":"Singing & Choral Groups",
    "A6C":"Bands & Ensembles",
    "A6E":"Performing Arts Schools",
    "A70":"Humanities",
    "A80":"Historical Organizations",
    "A82":"Historical Societies & Historic Preservation",
    "A84":"Commemorative Events",
    "A90":"Arts Services",
    "A99":"Arts, Culture & Humanities N.E.C.",
    "B01":"Alliances & Advocacy",
    "B02":"Management & Technical Assistance",
    "B03":"Professional Societies & Associations",
    "B05":"Research Institutes & Public Policy Analysis",
    "B11":"Single Organization Support",
    "B12":"Fund Raising & Fund Distribution",
    "B19":"Support N.E.C.",
    "B20":"Elementary & Secondary Schools",
    "B21":"Preschools",
    "B24":"Primary & Elementary Schools",
    "B25":"Secondary & High Schools",
    "B28":"Special Education",
    "B29":"Charter Schools",
    "B30":"Vocational & Technical Schools",
    "B40":"Higher Education",
    "B41":"Two-Year Colleges",
    "B42":"Undergraduate Colleges",
    "B43":"Universities",
    "B50":"Graduate & Professional Schools",
    "B60":"Adult Education",
    "B70":"Libraries",
    "B80":"Student Services",
    "B82":"Scholarships & Student Financial Aid",
    "B83":"Student Sororities & Fraternities",
    "B84":"Alumni Associations",
    "B90":"Educational Services",
    "B92":"Remedial Reading & Encouragement",
    "B94":"Parent & Teacher Groups",
    "B99":"Education N.E.C.",
    "C01":"Alliances & Advocacy",
    "C02":"Management & Technical Assistance",
    "C03":"Professional Societies & Associations",
    "C05":"Research Institutes & Public Policy Analysis",
    "C11":"Single Organization Support",
    "C12":"Fund Raising & Fund Distribution",
    "C19":"Support N.E.C.",
    "C20":"Pollution Abatement & Control",
    "C27":"Recycling",
    "C30":"Natural Resources Conservation & Protection",
    "C32":"Water Resources, Wetlands Conservation & Management",
    "C34":"Land Resources Conservation",
    "C35":"Energy Resources Conservation & Development",
    "C36":"Forest Conservation",
    "C40":"Botanical, Horticultural & Landscape Services",
    "C41":"Botanical Gardens & Arboreta",
    "C42":"Garden Clubs",
    "C50":"Environmental Beautification",
    "C60":"Environmental Education",
    "C99":"Environment N.E.C.",
    "D01":"Alliances & Advocacy",
    "D02":"Management & Technical Assistance",
    "D03":"Professional Societies & Associations",
    "D05":"Research Institutes & Public Policy Analysis",
    "D11":"Single Organization Support",
    "D12":"Fund Raising & Fund Distribution",
    "D19":"Support N.E.C.",
    "D20":"Animal Protection & Welfare",
    "D30":"Wildlife Preservation & Protection",
    "D31":"Protection of Endangered Species",
    "D32":"Bird Sanctuaries",
    "D33":"Fisheries Resources",
    "D34":"Wildlife Sanctuaries",
    "D40":"Veterinary Services",
    "D50":"Zoos & Aquariums",
    "D60":"Animal Services N.E.C.",
    "D61":"Animal Training",
    "D99":"Animal-Related N.E.C.",
    "E01":"Alliances & Advocacy",
    "E02":"Management & Technical Assistance",
    "E03":"Professional Societies & Associations",
    "E05":"Research Institutes & Public Policy Analysis",
    "E11":"Single Organization Support",
    "E12":"Fund Raising & Fund Distribution",
    "E19":"Support N.E.C.",
    "E20":"Hospitals",
    "E21":"Community Health Systems",
    "E22":"General Hospitals",
    "E24":"Specialty Hospitals",
    "E30":"Ambulatory & Primary Health Care",
    "E31":"Group Health Practices",
    "E32":"Community Clinics",
    "E40":"Reproductive Health Care",
    "E42":"Family Planning",
    "E50":"Rehabilitative Care",
    "E60":"Health Support",
    "E61":"Blood Banks",
    "E62":"Emergency Medical Services & Transport",
    "E65":"Organ & Tissue Banks",
    "E6A":"Pharmacies and drug stores",
    "E70":"Public Health",
    "E80":"Health (General & Financing)",
    "E86":"Patient & Family Support",
    "E90":"Nursing",
    "E91":"Nursing Facilities",
    "E92":"Home Health Care",
    "E99":"Health Care N.E.C.",
    "F01":"Alliances & Advocacy",
    "F02":"Management & Technical Assistance",
    "F03":"Professional Societies & Associations",
    "F05":"Research Institutes & Public Policy Analysis",
    "F11":"Single Organization Support",
    "F12":"Fund Raising & Fund Distribution",
    "F19":"Support N.E.C.",
    "F20":"Substance Abuse Dependency, Prevention & Treatment",
    "F21":"Substance Abuse Prevention",
    "F22":"Substance Abuse Treatment",
    "F30":"Mental Health Treatment",
    "F31":"Psychiatric Hospitals",
    "F32":"Community Mental Health Centers",
    "F33":"Residential Mental Health Treatment",
    "F40":"Hot Lines & Crisis Intervention",
    "F42":"Sexual Assault Services",
    "F50":"Addictive Disorders N.E.C.",
    "F52":"Smoking Addiction",
    "F53":"Eating Disorders & Addictions",
    "F54":"Gambling Addiction",
    "F60":"Counseling",
    "F70":"Mental Health Disorders",
    "F80":"Mental Health Associations",
    "F99":"Mental Health & Crisis Intervention N.E.C.",
    "G01":"Alliances & Advocacy",
    "G02":"Management & Technical Assistance",
    "G03":"Professional Societies & Associations",
    "G05":"Research Institutes & Public Policy Analysis",
    "G11":"Single Organization Support",
    "G12":"Fund Raising & Fund Distribution",
    "G19":"Support N.E.C.",
    "G20":"Birth Defects & Genetic Diseases",
    "G25":"Down Syndrome",
    "G30":"Cancer",
    "G32":"Breast Cancer",
    "G40":"Diseases of Specific Organs",
    "G41":"Eye Diseases, Blindness & Vision Impairments",
    "G42":"Ear & Throat Diseases",
    "G43":"Heart & Circulatory System Diseases & Disorders",
    "G44":"Kidney Diseases",
    "G45":"Lung Diseases",
    "G48":"Brain Disorders",
    "G50":"Nerve, Muscle & Bone Diseases",
    "G51":"Arthritis",
    "G54":"Epilepsy",
    "G60":"Allergy-Related Diseases",
    "G61":"Asthma",
    "G70":"Digestive Diseases & Disorders",
    "G80":"Specifically Named Diseases",
    "G81":"AIDS",
    "G83":"Alzheimers Disease",
    "G84":"Autism",
    "G90":"Medical Disciplines",
    "G92":"Biomedicine & Bioengineering",
    "G94":"Geriatrics",
    "G96":"Neurology & Neuroscience",
    "G98":"Pediatrics",
    "G99":"Voluntary Health Associations & Medical Disciplines N.E.C.",
    "G9B":"Surgical Specialties",
    "H01":"Alliances & Advocacy",
    "H02":"Management & Technical Assistance",
    "H03":"Professional Societies & Associations",
    "H05":"Research Institutes & Public Policy Analysis",
    "H11":"Single Organization Support",
    "H12":"Fund Raising & Fund Distribution",
    "H19":"Support N.E.C.",
    "H20":"Birth Defects & Genetic Diseases Research",
    "H25":"Down Syndrome Research",
    "H30":"Cancer Research",
    "H32":"Breast Cancer Research",
    "H40":"Diseases of Specific Organs Research",
    "H41":"Eye Diseases, Blindness & Vision Impairments Research",
    "H42":"Ear & Throat Diseases Research",
    "H43":"Heart & Circulatory System Diseases & Disorders Research",
    "H44":"Kidney Diseases Research",
    "H45":"Lung Diseases Research",
    "H48":"Brain Disorders Research",
    "H50":"Nerve, Muscle & Bone Diseases Research",
    "H51":"Arthritis Research",
    "H54":"Epilepsy Research",
    "H60":"Allergy-Related Diseases Research",
    "H61":"Asthma Research",
    "H70":"Digestive Diseases & Disorders Research",
    "H80":"Specifically Named Diseases Research",
    "H81":"AIDS Research",
    "H83":"Alzheimers Disease Research",
    "H84":"Autism Research",
    "H90":"Medical Disciplines Research",
    "H92":"Biomedicine & Bioengineering Research",
    "H94":"Geriatrics Research",
    "H96":"Neurology & Neuroscience Research",
    "H98":"Pediatrics Research",
    "H99":"Medical Research N.E.C.",
    "H9B":"Surgical Specialties Research",
    "I01":"Alliances & Advocacy",
    "I02":"Management & Technical Assistance",
    "I03":"Professional Societies & Associations",
    "I05":"Research Institutes & Public Policy Analysis",
    "I11":"Single Organization Support",
    "I12":"Fund Raising & Fund Distribution",
    "I19":"Support N.E.C.",
    "I20":"Crime Prevention",
    "I21":"Youth Violence Prevention",
    "I23":"Drunk Driving-Related",
    "I30":"Correctional Facilities",
    "I31":"Half-Way Houses for Offenders & Ex-Offenders",
    "I40":"Rehabilitation Services for Offenders",
    "I43":"Inmate Support",
    "I44":"Prison Alternatives",
    "I50":"Administration of Justice",
    "I51":"Dispute Resolution & Mediation",
    "I60":"Law Enforcement",
    "I70":"Protection Against Abuse",
    "I71":"Spouse Abuse Prevention",
    "I72":"Child Abuse Prevention",
    "I73":"Sexual Abuse Prevention",
    "I80":"Legal Services",
    "I83":"Public Interest Law",
    "I99":"Crime & Legal-Related N.E.C.",
    "J01":"Alliances & Advocacy",
    "J02":"Management & Technical Assistance",
    "J03":"Professional Societies & Associations",
    "J05":"Research Institutes & Public Policy Analysis",
    "J11":"Single Organization Support",
    "J12":"Fund Raising & Fund Distribution",
    "J19":"Support N.E.C.",
    "J20":"Employment Preparation & Procurement",
    "J21":"Vocational Counseling",
    "J22":"Job Training",
    "J30":"Vocational Rehabilitation",
    "J32":"Goodwill Industries",
    "J33":"Sheltered Employment",
    "J40":"Labor Unions",
    "J99":"Employment N.E.C.",
    "K01":"Alliances & Advocacy",
    "K02":"Management & Technical Assistance",
    "K03":"Professional Societies & Associations",
    "K05":"Research Institutes & Public Policy Analysis",
    "K11":"Single Organization Support",
    "K12":"Fund Raising & Fund Distribution",
    "K19":"Support N.E.C.",
    "K20":"Agricultural Programs",
    "K25":"Farmland Preservation",
    "K26":"Animal Husbandry",
    "K28":"Farm Bureaus & Granges",
    "K2A":"Other Vegetable (except Potato) and Melon Farming",
    "K2B":"Soil Preparation, Planting, and Cultivating",
    "K2C":"Wineries",
    "K30":"Food Programs",
    "K31":"Food Banks & Pantries",
    "K34":"Congregate Meals",
    "K35":"Soup Kitchens",
    "K36":"Meals on Wheels",
    "K40":"Nutrition",
    "K50":"Home Economics",
    "K6A":"Meat markets",
    "K6B":"Confectionery and Nut Stores",
    "K6C":"Caterers",
    "K6D":"Mobile food services",
    "K6E":"Drinking Places (Alcoholic Beverages)",
    "K6F":"Snack and Nonalcoholic Beverage Bars",
    "K90":"Limited-Service Restaurants",
    "K91":"Supermarkets and Other Grocery (except Convenience) Stores",
    "K92":"Convenience Stores",
    "K93":"Fruit and Vegetable Markets",
    "K94":"All Other Specialty Food Stores",
    "K95":"Food (Health) Supplement Stores",
    "K96":"Warehouse Clubs and Supercenters",
    "K97":"Food Service Contractors",
    "K98":"Full-Service Restaurants",
    "K99":"Food, Agriculture & Nutrition N.E.C.",
    "L01":"Alliances & Advocacy",
    "L02":"Management & Technical Assistance",
    "L03":"Professional Societies & Associations",
    "L05":"Research Institutes & Public Policy Analysis",
    "L11":"Single Organization Support",
    "L12":"Fund Raising & Fund Distribution",
    "L19":"Support N.E.C.",
    "L20":"Housing Development, Construction & Management",
    "L21":"Low-Income & Subsidized Rental Housing",
    "L22":"Senior Citizens Housing & Retirement Communities",
    "L24":"Independent Housing for People with Disabilities",
    "L25":"Housing Rehabilitation",
    "L30":"Housing Search Assistance",
    "L40":"Temporary Housing",
    "L41":"Homeless Shelters",
    "L4A":"Hotels (except Casino Hotels) and Motels",
    "L4B":"Bed-and-breakfast inns",
    "L50":"Homeowners & Tenants Associations",
    "L80":"Housing Support",
    "L81":"Home Improvement & Repairs",
    "L82":"Housing Expense Reduction Support",
    "L99":"Housing & Shelter N.E.C.",
    "M01":"Alliances & Advocacy",
    "M02":"Management & Technical Assistance",
    "M03":"Professional Societies & Associations",
    "M05":"Research Institutes & Public Policy Analysis",
    "M11":"Single Organization Support",
    "M12":"Fund Raising & Fund Distribution",
    "M19":"Support N.E.C.",
    "M20":"Disaster Preparedness & Relief Services",
    "M23":"Search & Rescue Squads",
    "M24":"Fire Prevention",
    "M40":"Safety Education",
    "M41":"First Aid",
    "M42":"Automotive Safety",
    "M60":"Public Safety Benevolent Associations",
    "M99":"Public Safety, Disaster Preparedness & Relief N.E.C.",
    "N01":"Alliances & Advocacy",
    "N02":"Management & Technical Assistance",
    "N03":"Professional Societies & Associations",
    "N05":"Research Institutes & Public Policy Analysis",
    "N11":"Single Organization Support",
    "N12":"Fund Raising & Fund Distribution",
    "N19":"Support N.E.C.",
    "N20":"Camps",
    "N2A":"RV (Recreational Vehicle) Parks and Campgrounds",
    "N2B":"Recreational and Vacation Camps (Except Campgrounds)",
    "N30":"Physical Fitness & Community Recreational Facilities",
    "N31":"Community Recreational Centers",
    "N32":"Parks & Playgrounds",
    "N40":"Sports Associations & Training Facilities",
    "N50":"Recreational Clubs",
    "N52":"Fairs",
    "N60":"Amateur Sports",
    "N61":"Fishing & Hunting",
    "N62":"Basketball",
    "N63":"Baseball & Softball",
    "N64":"Soccer",
    "N65":"Football",
    "N66":"Racquet Sports",
    "N67":"Swimming & Other Water Recreation",
    "N68":"Winter Sports",
    "N69":"Equestrian",
    "N6A":"Golf",
    "N70":"Amateur Sports Competitions",
    "N71":"Olympics",
    "N72":"Special Olympics",
    "N80":"Professional Athletic Leagues",
    "N99":"Recreation & Sports N.E.C.",
    "O01":"Alliances & Advocacy",
    "O02":"Management & Technical Assistance",
    "O03":"Professional Societies & Associations",
    "O05":"Research Institutes & Public Policy Analysis",
    "O11":"Single Organization Support",
    "O12":"Fund Raising & Fund Distribution",
    "O19":"Support N.E.C.",
    "O20":"Youth Centers & Clubs",
    "O21":"Boys Clubs",
    "O22":"Girls Clubs",
    "O23":"Boys & Girls Clubs",
    "O30":"Adult & Child Matching Programs",
    "O31":"Big Brothers & Big Sisters",
    "O40":"Scouting",
    "O41":"Boy Scouts of America",
    "O42":"Girl Scouts of the U.S.A.",
    "O43":"Camp Fire",
    "O50":"Youth Development Programs",
    "O51":"Youth Community Service Clubs",
    "O52":"Youth Development - Agricultural",
    "O53":"Youth Development - Business",
    "O54":"Youth Development - Citizenship",
    "O55":"Youth Development - Religious Leadership",
    "O99":"Youth Development N.E.C.",
    "P01":"Alliances & Advocacy",
    "P02":"Management & Technical Assistance",
    "P03":"Professional Societies & Associations",
    "P05":"Research Institutes & Public Policy Analysis",
    "P11":"Single Organization Support",
    "P12":"Fund Raising & Fund Distribution",
    "P19":"Support N.E.C.",
    "P20":"Human Service Organizations",
    "P21":"American Red Cross",
    "P22":"Urban League",
    "P24":"Salvation Army",
    "P26":"Volunteers of America",
    "P27":"Young Mens or Womens Associations",
    "P28":"Neighborhood Centers",
    "P29":"Thrift Shops",
    "P30":"Children & Youth Services",
    "P31":"Adoption",
    "P32":"Foster Care",
    "P33":"Child Day Care",
    "P40":"Family Services",
    "P42":"Single Parent Agencies",
    "P43":"Family Violence Shelters",
    "P44":"In-Home Assistance",
    "P45":"Family Services for Adolescent Parents",
    "P46":"Family Counseling",
    "P47":"Pregnancy Centers",
    "P50":"Personal Social Services",
    "P51":"Financial Counseling",
    "P52":"Transportation Assistance",
    "P58":"Gift Distribution",
    "P60":"Emergency Assistance",
    "P61":"Travelers Aid",
    "P62":"Victims Services",
    "P70":"Residential Care & Adult Day Programs",
    "P71":"Adult Day Care",
    "P73":"Group Homes",
    "P74":"Hospices",
    "P75":"Supportive Housing for Older Adults",
    "P76":"Homes for Children & Adolescents",
    "P7A":"Residential Intellectual and Developmental Disability Facilities (group homes, intermediate care facilities & hospitals)",
    "P80":"Centers to Support the Independence of Specific Populations",
    "P81":"Senior Centers",
    "P82":"Developmentally Disabled Centers",
    "P83":"Women's Centers",
    "P84":"Ethnic & Immigrant Centers",
    "P85":"Homeless Centers",
    "P86":"Blind & Visually Impaired Centers",
    "P87":"Deaf & Hearing Impaired Centers",
    "P88":"LGBT Centers",
    "P99":"Human Services N.E.C.",
    "Q01":"Alliances & Advocacy",
    "Q02":"Management & Technical Assistance",
    "Q03":"Professional Societies & Associations",
    "Q05":"Research Institutes & Public Policy Analysis",
    "Q11":"Single Organization Support",
    "Q12":"Fund Raising & Fund Distribution",
    "Q19":"Support N.E.C.",
    "Q20":"Promotion of International Understanding",
    "Q21":"International Cultural Exchange",
    "Q22":"International Academic Exchange",
    "Q23":"International Exchange N.E.C.",
    "Q30":"International Development",
    "Q31":"International Agricultural Development",
    "Q32":"International Economic Development",
    "Q33":"International Relief",
    "Q34":"International Educational Development ",
    "Q35":"International Democracy & Civil Society Development",
    "Q36":"International Science & Technology Development ",
    "Q38":"International Environment, Population & Sustainability ",
    "Q39":"International Health Development ",
    "Q40":"International Peace & Security",
    "Q41":"Arms Control & Peace",
    "Q42":"United Nations Associations",
    "Q43":"National Security",
    "Q50":"International Affairs, Foreign Policy, & Globalization",
    "Q51":"International Economic & Trade Policy",
    "Q70":"International Human Rights",
    "Q71":"International Migration & Refugee Issues",
    "Q99":"International, Foreign Affairs & National Security N.E.C.",
    "R01":"Alliances & Advocacy",
    "R02":"Management & Technical Assistance",
    "R03":"Professional Societies & Associations",
    "R05":"Research Institutes & Public Policy Analysis",
    "R11":"Single Organization Support",
    "R12":"Fund Raising & Fund Distribution",
    "R19":"Support N.E.C.",
    "R20":"Civil Rights",
    "R21":"Immigrants' Rights",
    "R22":"Minority Rights",
    "R23":"Disabled Persons Rights",
    "R24":"Womens Rights",
    "R25":"Seniors Rights",
    "R26":"Lesbian & Gay Rights",
    "R27":"Patients' Rights",
    "R28":"Children's Rights",
    "R29":"Employee & Workers' Rights",
    "R30":"Intergroup & Race Relations",
    "R40":"Voter Education & Registration",
    "R60":"Civil Liberties",
    "R61":"Reproductive Rights",
    "R62":"Right to Life",
    "R63":"Censorship, Freedom of Speech & Press",
    "R65":"Freedom of Religion Issues",
    "R67":"Right to Die & Euthanasia",
    "R99":"Civil Rights, Social Action & Advocacy N.E.C.",
    "S01":"Alliances & Advocacy",
    "S02":"Management & Technical Assistance",
    "S03":"Professional Societies & Associations",
    "S05":"Research Institutes & Public Policy Analysis",
    "S11":"Single Organization Support",
    "S12":"Fund Raising & Fund Distribution",
    "S19":"Support N.E.C.",
    "S20":"Community & Neighborhood Development",
    "S21":"Community Coalitions",
    "S22":"Neighborhood & Block Associations",
    "S30":"Economic Development",
    "S31":"Urban & Community Economic Development",
    "S32":"Rural Economic Development",
    "S40":"Business & Industry",
    "S41":"Chambers of Commerce & Business Leagues",
    "S43":"Small Business Development",
    "S46":"Boards of Trade",
    "S47":"Real Estate Associations",
    "S50":"Nonprofit Management",
    "S80":"Community Service Clubs",
    "S81":"Womens Service Clubs",
    "S82":"Mens Service Clubs",
    "S99":"Community Improvement & Capacity Building N.E.C.",
    "T01":"Alliances & Advocacy",
    "T02":"Management & Technical Assistance",
    "T03":"Professional Societies & Associations",
    "T05":"Research Institutes & Public Policy Analysis",
    "T11":"Single Organization Support",
    "T12":"Fund Raising & Fund Distribution",
    "T19":"Support N.E.C.",
    "T20":"Private Grantmaking Foundations",
    "T21":"Corporate Foundations",
    "T22":"Private Independent Foundations",
    "T23":"Private Operating Foundations",
    "T30":"Public Foundations",
    "T31":"Community Foundations",
    "T40":"Voluntarism Promotion",
    "T50":"Philanthropy, Charity & Voluntarism Promotion",
    "T70":"Federated Giving Programs",
    "T90":"Named Trusts N.E.C.",
    "T99":"Philanthropy, Voluntarism & Grantmaking Foundations N.E.C.",
    "U01":"Alliances & Advocacy",
    "U02":"Management & Technical Assistance",
    "U03":"Professional Societies & Associations",
    "U05":"Research Institutes & Public Policy Analysis",
    "U11":"Single Organization Support",
    "U12":"Fund Raising & Fund Distribution",
    "U19":"Support N.E.C.",
    "U20":"General Science",
    "U21":"Marine Science & Oceanography",
    "U30":"Physical & Earth Sciences",
    "U31":"Astronomy",
    "U33":"Chemistry & Chemical Engineering",
    "U34":"Mathematics",
    "U36":"Geology",
    "U40":"Engineering & Technology",
    "U41":"Computer Science",
    "U42":"Engineering",
    "U50":"Biological & Life Sciences",
    "U99":"Science & Technology N.E.C.",
    "V01":"Alliances & Advocacy",
    "V02":"Management & Technical Assistance",
    "V03":"Professional Societies & Associations",
    "V05":"Research Institutes & Public Policy Analysis",
    "V11":"Single Organization Support",
    "V12":"Fund Raising & Fund Distribution",
    "V19":"Support N.E.C.",
    "V20":"Social Science",
    "V21":"Anthropology & Sociology",
    "V22":"Economics",
    "V23":"Behavioral Science",
    "V24":"Political Science",
    "V25":"Population Studies",
    "V26":"Law & Jurisprudence",
    "V30":"Interdisciplinary Research",
    "V31":"Black Studies",
    "V32":"Womens Studies",
    "V33":"Ethnic Studies",
    "V34":"Urban Studies",
    "V35":"International Studies",
    "V36":"Gerontology",
    "V37":"Labor Studies",
    "V99":"Social Science N.E.C.",
    "W01":"Alliances & Advocacy",
    "W02":"Management & Technical Assistance",
    "W03":"Professional Societies & Associations",
    "W05":"Research Institutes & Public Policy Analysis",
    "W11":"Single Organization Support",
    "W12":"Fund Raising & Fund Distribution",
    "W19":"Support N.E.C.",
    "W20":"Government & Public Administration",
    "W22":"Public Finance, Taxation & Monetary Policy",
    "W24":"Citizen Participation",
    "W30":"Military & Veterans Organizations",
    "W40":"Public Transportation Systems",
    "W50":"Telecommunications",
    "W60":"Financial Institutions",
    "W61":"Credit Unions",
    "W70":"Leadership Development",
    "W80":"Public Utilities",
    "W90":"Consumer Protection",
    "W99":"Public & Societal Benefit N.E.C.",
    "X01":"Alliances & Advocacy",
    "X02":"Management & Technical Assistance",
    "X03":"Professional Societies & Associations",
    "X05":"Research Institutes & Public Policy Analysis",
    "X11":"Single Organization Support",
    "X12":"Fund Raising & Fund Distribution",
    "X19":"Support N.E.C.",
    "X20":"Christianity",
    "X21":"Protestant",
    "X22":"Roman Catholic",
    "X30":"Judaism",
    "X40":"Islam",
    "X50":"Buddhism",
    "X70":"Hinduism",
    "X80":"Religious Media & Communications",
    "X81":"Religious Film & Video",
    "X82":"Religious Television",
    "X83":"Religious Printing & Publishing",
    "X84":"Religious Radio",
    "X90":"Interfaith Coalitions",
    "X99":"Religion-Related N.E.C.",
    "Y01":"Alliances & Advocacy",
    "Y02":"Management & Technical Assistance",
    "Y03":"Professional Societies & Associations",
    "Y05":"Research Institutes & Public Policy Analysis",
    "Y11":"Single Organization Support",
    "Y12":"Fund Raising & Fund Distribution",
    "Y19":"Support N.E.C.",
    "Y20":"Insurance Providers",
    "Y22":"Local Benevolent Life Insurance Associations",
    "Y23":"Mutual Insurance Companies & Associations",
    "Y24":"Supplemental Unemployment Compensation",
    "Y25":"State-Sponsored Workers Compensation Reinsurance Organizations",
    "Y30":"Pension & Retirement Funds",
    "Y33":"Teachers Retirement Fund Associations",
    "Y34":"Employee Funded Pension Trusts",
    "Y35":"Multi-Employer Pension Plans",
    "Y40":"Fraternal Societies",
    "Y41":"Fraternal Beneficiary Societies",
    "Y42":"Domestic Fraternal Societies",
    "Y43":"Voluntary Employees Beneficiary Associations (Non-Government)",
    "Y44":"Voluntary Employees Beneficiary Associations Government)",
    "Y50":"Cemeteries",
    "Y99":"Mutual & Membership Benefit N.E.C.",
    "Z99":"Unknown"
    }

In [12]:
# Read BMF Data

processed_chunks = []

chunksize = 1000

for chunk in pd.read_csv(filepath+'bmf.csv', chunksize=chunksize, na_filter=False):
    
    chunk = chunk[['EIN', 'ORG_NAME_CURRENT', 'F990_ORG_ADDR_ZIP', 'NTEEV2']]

    chunk = chunk[(chunk != '').all(axis=1)]
    
    chunk['EIN'] = chunk['EIN'].astype(int)
    chunk['F990_ZIP'] = chunk['F990_ORG_ADDR_ZIP'].astype(str).str[:5].astype(int)
    chunk[['IndustryGroup', 'MajorGroup', 'OrganizationType']] = chunk['NTEEV2'].str.split('-', expand=True)
    chunk['IndustryGroup'] = chunk['IndustryGroup'].map(industry_map)
    chunk['OrganizationType'] = chunk['OrganizationType'].map(organization_map)
    chunk['MajorGroup'] = chunk['MajorGroup'].map(major_map)

    chunk = chunk[['EIN', 'ORG_NAME_CURRENT', 'F990_ZIP', 'IndustryGroup', 'MajorGroup', 'OrganizationType']]
    
    processed_chunks.append(chunk)

bmf_df = pd.concat(processed_chunks)

bmf_df.dropna(inplace=True)
bmf_df = bmf_df.applymap(lambda x: x.upper().strip() if isinstance(x, str) else x)
bmf_df.drop_duplicates(inplace=True)

In [13]:
#save NTEE Details in a separate file

# bmf_df.to_csv(filepath+'bmf_codes.csv', index=False)

In [23]:
bmf_df.head()

,EIN,ORG_NAME_CURRENT,F990_ZIP,IndustryGroup,MajorGroup,OrganizationType
1,1,VOLUNTEERS OF AMERICA INC,63130,UNIVERSITIES,UNIVERSITIES,REGULAR NONPROFIT
2,4,SCRIPT INC,43216,EDUCATION,EDUCATIONAL SERVICES,REGULAR NONPROFIT
3,5,JEKYLL ISLAND MUSEUM ASSOCIATES INC,31527,"ARTS, CULTURE, AND HUMANITIES",MUSEUMS,REGULAR NONPROFIT
4,3154,OAKLEAF FOREST TENANT MANAGEMENT,23523,ENVIRONMENT AND ANIMALS,FOREST CONSERVATION,REGULAR NONPROFIT
5,4101,SOUTH LAFOURCHE QUARTERBACK CLUB,70374,HUMAN SERVICES,FOOTBALL,REGULAR NONPROFIT


In [26]:
recipient_df = bmf_df.copy()

recipient_df = recipient_df.groupby(['ORG_NAME_CURRENT', 'F990_ZIP']).agg({ 
    'EIN': 'max',
    'IndustryGroup': lambda x: max(set(x), key=lambda y: list(x).count(y)),
    'MajorGroup': lambda x: max(set(x), key=lambda y: list(x).count(y)),
    'OrganizationType': lambda x: max(set(x), key=lambda y: list(x).count(y)),
}).reset_index()

recipient_df = recipient_df.rename(
                    columns={'EIN': 'RecipientEIN',
                             'ORG_NAME_CURRENT':'RecipientName', 
                             'F990_ZIP':'RecipientZIP',
                             'IndustryGroup':'RecipientIndustryGroup',
                             'MajorGroup':'RecipientMajorGroup',
                             'OrganizationType':'RecipientOrganizationType'
                             })



grantmaker_df = bmf_df.copy()
grantmaker_df.drop(columns=['ORG_NAME_CURRENT'], axis=1, inplace=True)
grantmaker_df = grantmaker_df.rename(
                    columns={'EIN': 'EIN', 
                             'F990_ZIP':'ZIP',
                             'IndustryGroup':'GrantmakerIndustryGroup',
                             'MajorGroup':'GrantmakerMajorGroup',
                             'OrganizationType':'GrantmakerOrganizationType'
                             })

grantmaker_df = grantmaker_df.groupby(['EIN', 'ZIP']).agg({
                                'GrantmakerIndustryGroup': lambda x: max(set(x), key=lambda y: list(x).count(y)),
                                'GrantmakerMajorGroup': lambda x: max(set(x), key=lambda y: list(x).count(y)),
                                'GrantmakerOrganizationType': lambda x: max(set(x), key=lambda y: list(x).count(y)),
                            }).reset_index()



In [27]:
Form_990PF_RECP = pd.merge(Form_990PF, recipient_df, on=['RecipientName','RecipientZIP'], how='left')
Form_990PF_COMP = pd.merge(Form_990PF_RECP, grantmaker_df, on=['EIN','ZIP'], how='left')

Form_990_RECP = pd.merge(Form_990, recipient_df[['RecipientZIP', 'RecipientEIN', 'RecipientIndustryGroup', 'RecipientMajorGroup', 'RecipientOrganizationType']], on=['RecipientEIN', 'RecipientZIP'], how='left')
Form_990_COMP = pd.merge(Form_990_RECP, grantmaker_df[['EIN', 'ZIP', 'GrantmakerIndustryGroup', 'GrantmakerMajorGroup', 'GrantmakerOrganizationType']], on=['EIN','ZIP'], how='left')

In [28]:
eo = pd.read_csv(filepath+'eo.csv', header=0, na_filter=False)

In [29]:
eo = eo[['EIN', 'NTEE_CD', 'ZIP', 'NAME']]
eo = eo[(eo != '').all(axis=1)]

In [30]:
eo['NTEE_CD'] = eo['NTEE_CD'].str[:3].map(major_map)

eo['ZIP'] = eo['ZIP'].str[:5].astype(int)
eo.dropna(inplace=True)
eo = eo.applymap(lambda x: x.upper().strip() if isinstance(x, str) else x)

eo.head()

,EIN,NTEE_CD,ZIP,NAME
2,587764,PROTESTANT,1852,IGLESIA BETHESDA INC
3,635913,PROTESTANT,1840,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC
4,765634,CHRISTIANITY,2126,MERCY CHAPEL INTERNATIONAL
5,841363,CHRISTIANITY,2126,AGAPE HOUSE OF PRAYER
6,852649,CHRISTIANITY,2445,BETHANY PRESBYTERIAN CHURCH


In [31]:
gm_eo = eo.copy()
gm_eo.rename(columns = {'NTEE_CD':'GrantmakerNTEEType'}, inplace=True)
gm_eo.head()

,EIN,GrantmakerNTEEType,ZIP,NAME
2,587764,PROTESTANT,1852,IGLESIA BETHESDA INC
3,635913,PROTESTANT,1840,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC
4,765634,CHRISTIANITY,2126,MERCY CHAPEL INTERNATIONAL
5,841363,CHRISTIANITY,2126,AGAPE HOUSE OF PRAYER
6,852649,CHRISTIANITY,2445,BETHANY PRESBYTERIAN CHURCH


In [32]:
rp_eo = eo.copy()
rp_eo.rename(columns={'EIN':'RecipientEIN', 'NTEE_CD':'RecipientNTEEType', 'NAME': 'RecipientName', 'ZIP':'RecipientZIP'}, inplace=True)
rp_eo.head()

,RecipientEIN,RecipientNTEEType,RecipientZIP,RecipientName
2,587764,PROTESTANT,1852,IGLESIA BETHESDA INC
3,635913,PROTESTANT,1840,MINISTERIO APOSTOLICO JESUCRISTO ES EL SENOR INC
4,765634,CHRISTIANITY,2126,MERCY CHAPEL INTERNATIONAL
5,841363,CHRISTIANITY,2126,AGAPE HOUSE OF PRAYER
6,852649,CHRISTIANITY,2445,BETHANY PRESBYTERIAN CHURCH


In [33]:
Form_990_COMP = pd.merge(Form_990_COMP, gm_eo[['EIN', 'GrantmakerNTEEType']], on=['EIN'], how='left')
Form_990_COMP = pd.merge(Form_990_COMP, rp_eo[['RecipientEIN', 'RecipientNTEEType']], on=['RecipientEIN'], how='left')

In [34]:
gm_eo = gm_eo[['EIN', 'GrantmakerNTEEType']].drop_duplicates()

rp_eo = rp_eo[['RecipientName','RecipientZIP', 'RecipientNTEEType']].drop_duplicates()
rp_eo = rp_eo.groupby(['RecipientName', 'RecipientZIP']).agg({
                                'RecipientNTEEType': lambda x: max(set(x), key=lambda y: list(x).count(y)),
                            }).reset_index()

In [35]:
Form_990PF_COMP = pd.merge(Form_990PF_COMP, gm_eo, on=['EIN'], how='left')
Form_990PF_COMP = pd.merge(Form_990PF_COMP, rp_eo, on=['RecipientName','RecipientZIP'], how='left')

In [36]:
Form_990PF_COMP = Form_990PF_COMP.reindex(columns=['FormType',
 'EIN',
 'TaxYear',
 'BusinessName',
 'Address',
 'City',
 'State',
 'ZIP',
 'GrantmakerIndustryGroup',
 'GrantmakerMajorGroup',
 'GrantmakerOrganizationType',
 'GrantmakerNTEEType',
 'TotalRevenue',
 'TotalExpenses',
 'TotalAssets',
 'TotalLiabilities',
 'GrantsAndAllocations',
 'RecipientName',
 'RecipientEIN',
 'RecipientAddress',
 'RecipientCity',
 'RecipientState',
 'RecipientZIP',
 'RecipientIndustryGroup',
 'RecipientMajorGroup',
 'RecipientOrganizationType',
 'RecipientNTEEType',
 'GrantAmount',
 ])

Form_990_COMP = Form_990_COMP.reindex(columns=['FormType',
 'EIN',
 'TaxYear',
 'BusinessName',
 'Address',
 'City',
 'State',
 'ZIP',
 'GrantmakerIndustryGroup',
 'GrantmakerMajorGroup',
 'GrantmakerOrganizationType',
 'GrantmakerNTEEType',
 'TotalRevenue',
 'TotalExpenses',
 'TotalAssets',
 'TotalLiabilities',
 'GrantsAndAllocations',
 'RecipientName',
 'RecipientEIN',
 'RecipientAddress',
 'RecipientCity',
 'RecipientState',
 'RecipientZIP',
 'RecipientIndustryGroup',
 'RecipientMajorGroup',
 'RecipientOrganizationType',
 'RecipientNTEEType',
 'GrantAmount',
 ])

In [37]:
Form_990_Consolidated = pd.concat([Form_990_COMP, Form_990PF_COMP])

In [38]:
Form_990_Consolidated.shape

(734891, 28)

In [39]:
cbsa_msa = pd.read_csv(filepath+'ZIP_CBSA_MSA.csv', header=0)

# ZIP,CBSA,CBSA Title,Metropolitan/Micropolitan Statistical Area

gm_cbsa_msa = cbsa_msa.copy().rename(columns={'CBSA':'GrantmakerCBSA','CBSA Title':'GrantmakerCBSATitle','Metropolitan/Micropolitan Statistical Area':'GrantmakerMSAType'})

rp_cbsa_msa = cbsa_msa.copy().rename(columns={'ZIP':'RecipientZIP','CBSA':'RecipientCBSA','CBSA Title':'RecipientCBSATitle','Metropolitan/Micropolitan Statistical Area':'RecipientMSAType'})

In [40]:
cbsa_msa['ZIP'].duplicated().sum()

0

In [41]:
Form_990_Consolidated = pd.merge(Form_990_Consolidated, gm_cbsa_msa, on=['ZIP'], how='left')
Form_990_Consolidated = pd.merge(Form_990_Consolidated, rp_cbsa_msa, on=['RecipientZIP'], how='left')

In [42]:
Form_990_Consolidated.columns.to_list()

['FormType',
 'EIN',
 'TaxYear',
 'BusinessName',
 'Address',
 'City',
 'State',
 'ZIP',
 'GrantmakerIndustryGroup',
 'GrantmakerMajorGroup',
 'GrantmakerOrganizationType',
 'GrantmakerNTEEType',
 'TotalRevenue',
 'TotalExpenses',
 'TotalAssets',
 'TotalLiabilities',
 'GrantsAndAllocations',
 'RecipientName',
 'RecipientEIN',
 'RecipientAddress',
 'RecipientCity',
 'RecipientState',
 'RecipientZIP',
 'RecipientIndustryGroup',
 'RecipientMajorGroup',
 'RecipientOrganizationType',
 'RecipientNTEEType',
 'GrantAmount',
 'GrantmakerCBSA',
 'GrantmakerCBSATitle',
 'GrantmakerMSAType',
 'RecipientCBSA',
 'RecipientCBSATitle',
 'RecipientMSAType']

In [43]:
Form_990_Consolidated = Form_990_Consolidated.reindex(columns=[
 'FormType',
 'EIN',
 'TaxYear',
 'BusinessName',
 'Address',
 'City',
 'State',
 'ZIP',
 'GrantmakerCBSA',
 'GrantmakerCBSATitle',
 'GrantmakerMSAType',
 'GrantmakerIndustryGroup',
 'GrantmakerMajorGroup',
 'GrantmakerOrganizationType',
 'GrantmakerNTEEType',
 'TotalRevenue',
 'TotalExpenses',
 'TotalAssets',
 'TotalLiabilities',
 'GrantsAndAllocations',
 'RecipientName',
 'RecipientEIN',
 'RecipientAddress',
 'RecipientCity',
 'RecipientState',
 'RecipientZIP',
 'RecipientCBSA',
 'RecipientCBSATitle',
 'RecipientMSAType',
 'RecipientIndustryGroup',
 'RecipientMajorGroup',
 'RecipientOrganizationType',
 'RecipientNTEEType',
 'GrantAmount' 
 ])

In [44]:
Form_990_Consolidated.to_csv(filepath+'Form_990_Consolidated.csv', index=False)

In [45]:
Form_990_Consolidated = pd.read_csv(filepath+'Form_990_Consolidated.csv', header=0, na_filter=False)

In [46]:
import numpy as np

Form_990_Consolidated['GrantmakerMajorGroup'] = np.where(
    Form_990_Consolidated['GrantmakerMajorGroup'] != '',
    Form_990_Consolidated['GrantmakerMajorGroup'],
    Form_990_Consolidated['GrantmakerNTEEType']
)

Form_990_Consolidated['RecipientMajorGroup'] = np.where(
    Form_990_Consolidated['RecipientMajorGroup'] != '',
    Form_990_Consolidated['RecipientMajorGroup'],
    Form_990_Consolidated['RecipientNTEEType']
)

Form_990_Consolidated.drop(columns=['GrantmakerNTEEType', 'RecipientNTEEType'], inplace=True)

In [ ]:
Form_990_Consolidated.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 734891 entries, 0 to 734890
Data columns (total 32 columns):
 #   Column                      Non-Null Count   Dtype 
---  ------                      --------------   ----- 
 0   FormType                    734891 non-null  object
 1   EIN                         734891 non-null  int64 
 2   TaxYear                     734891 non-null  int64 
 3   BusinessName                734891 non-null  object
 4   Address                     734891 non-null  object
 5   City                        734891 non-null  object
 6   State                       734891 non-null  object
 7   ZIP                         734891 non-null  int64 
 8   GrantmakerCBSA              734891 non-null  object
 9   GrantmakerCBSATitle         734891 non-null  object
 10  GrantmakerMSAType           734891 non-null  object
 11  GrantmakerIndustryGroup     734891 non-null  object
 12  GrantmakerMajorGroup        734891 non-null  object
 13  GrantmakerOrganizationType  7

In [47]:
Form_990_Consolidated.to_csv(filepath+'Form_990_Consolidated.csv', index=False)

In [ ]:
(Form_990_Consolidated == '').sum()

FormType                           0
EIN                                0
TaxYear                            0
BusinessName                       0
Address                            0
City                               0
State                              0
ZIP                                0
GrantmakerCBSA                 58824
GrantmakerCBSATitle            58824
GrantmakerMSAType              58824
GrantmakerIndustryGroup       178324
GrantmakerMajorGroup           35256
GrantmakerOrganizationType    178324
TotalRevenue                       0
TotalExpenses                      0
TotalAssets                        0
TotalLiabilities                   0
GrantsAndAllocations               1
RecipientName                    949
RecipientEIN                  429306
RecipientAddress               10358
RecipientCity                  10168
RecipientState                 10128
RecipientZIP                       0
RecipientCBSA                  81033
RecipientCBSATitle             81033
R